# 서울 구별 원룸/오피스텔 전세매물 분석(feat. 직방)

In [ ]:
import pandas as pd
import pickle
from tqdm import tqdm
from glob import glob
import re
import cufflinks as cf
import plotly.express as px
cf.go_offline()

## Preprocess

**신규 column 생성**<br>

- random_location 분리
- approve_date to approve_year
- 반지하 여부

In [7]:
def preprocess(df_raw):
    room_info = df_raw[COLS]
    room_info = room_info.replace('서울특별시', '서울시')
    room_info = room_info[room_info['local1'] == '서울시'].reset_index(drop=True)
    return room_info

def unify_date_format(date):
    date_converted = re.sub('[^0-9]', '', date)
    return date_converted

def get_year_from_date(date):
    date = re.sub('[^0-9]', '', date)
    if len(date) < 4:
        return -1
    head = int(date[:4])
    if head > 2022:  # 1980.01.01을 80.01.01 이렇게 표현한 경우
        return int('19' + date[:2])
    elif head < 1900:  # 2002.01.01을 02.01.01 이렇게 표현한 경우
        return int('20' + date[:2])
    else:
        return int(date[:4])

In [ ]:
import coredotdata as cdd
cdd.download_dataset("1W517jrj")

In [5]:
df_raw = pd.read_csv('dataset/room_info_220925.csv')

In [11]:
COLS = ['item_id', 'user_no','sales_type','보증금액','월세금액','전용면적_m2','공급면적_m2','local1', 'local2', 'local3',
        'random_location', 'floor', 'updated_at','approve_date', 'residence_type']

In [12]:
room_info = preprocess(df_raw)
room_info['approve_year'] = room_info.apply(lambda x:get_year_from_date(x['approve_date']), axis=1)
room_info.loc[room_info['approve_year'] == -1, 'approve_year'] = \
    int(room_info[room_info['approve_year'] != -1]['approve_year'].mean())
room_info['random_lat'] = room_info.apply(lambda x:x['random_location'].split(',')[0], axis=1)
room_info['random_long'] = room_info.apply(lambda x:x['random_location'].split(',')[1], axis=1)
room_info['is_underground'] = 0
room_info.loc[room_info['floor'] == '반지하', 'is_underground'] = 1

In [18]:
# room_info.to_csv('data_preprocessed/room_info_221002.csv', index=False)

In [23]:
room_info.head()

,item_id,user_no,sales_type,보증금액,월세금액,전용면적_m2,공급면적_m2,local1,local2,local3,random_location,floor,updated_at,approve_date,residence_type,approve_year,random_lat,random_long,is_underground
0,33446567,5987034,월세,2000,130.0,52.69,57.70,서울시,서대문구,창천동,"37.55975011593247,126.93329296545063",1,2022-09-21 17:21:02,1990.01.16,다세대주택,1990,37.55975011593247,126.93329296545063,0
1,33203240,322799,전세,55000,NaN,29.29,39.67,서울시,서대문구,창천동,"37.55860942977413,126.92758585174795",NaN,2022-09-15 14:04:27,2022.07.08,다세대주택,2022,37.55860942977413,126.92758585174795,0
2,33181218,8485448,월세,5000,200.0,39.08,59.00,서울시,서대문구,창천동,"37.55857456629137,126.92824978616616",5,2022-09-14 15:12:46,2022-07-08,오피스텔,2022,37.55857456629137,126.92824978616616,0
3,33459584,5987034,월세,3000,165.0,72.73,79.34,서울시,마포구,연남동,"37.56050038416363,126.91826753775781",2,2022-09-22 16:21:05,1993-10-06,다가구용 단독주택,1993,37.56050038416363,126.91826753775781,0
4,32955508,17675823,전세,35000,NaN,29.75,39.67,서울시,마포구,창전동,"37.55300474509232,126.92640419917751",2,2022-09-01 21:22:07,20190820,제2종근린생활시설,2019,37.55300474509232,126.92640419917751,0


## EDA

### 서울 구별 매물 비율

In [48]:
# room_info['sales_type'].unique()  # ['월세', '전세']

In [32]:
count_by_gu = room_info.groupby('local2').size()  # 구별 매물 비율
charter_ratio = room_info.groupby('local2').apply(lambda x:sum(x['sales_type'] == '전세')/len(x))  # 구별 월세 비율

In [33]:
count_by_gu = count_by_gu.sort_values().reset_index()

In [34]:
count_by_gu.columns = ['local2', 'value']

In [35]:
count_by_gu.iplot(kind='pie', labels='local2', values='value', title='서울 구별 매물 비율')

- 서울의 전/월세 매물 수는 관악구-강남구-강서구... 순이다.

In [37]:
charter_ratio.sort_values().iplot(kind='bar', title="서울 구별 전세 비율")

- 그래프의 왼쪽에 있는 강남구, 강북구, 노원구 등은 월세의 비율이 높다.
- 반대로 오른쪽에 있는 양천구, 은평구, 강서구 등은 전세의 비율이 높다.

### 서울 전체 전세 보증금 분포

In [38]:
room_charter = room_info[room_info['sales_type'] == '전세'].reset_index(drop=True)

In [42]:
room_charter['보증금액'].iplot(kind='hist')

In [47]:
room_charter[room_charter['보증금액']< 50000]['보증금액'].iplot(kind='hist', bins=30)

- 원룸/오피스텔의 전세 보증금은 2억 전후를 평균으로 정규분포 형태를 따른다.

### 구별 전세보증금

In [53]:
room_charter['random_lat'] = room_charter['random_lat'].astype(float)
room_charter['random_long'] = room_charter['random_long'].astype(float)

In [ ]:
charter_ratio.sort_values().iplot(kind='bar', title="서울 구별 전세 비율")

In [64]:
room_charter.groupby('local2').mean('보증금액').sort_values('보증금액').iplot(kind='bar',
                                            y='보증금액', title='서울 구별 평균 전세보증금')

- 노원구의 평균 전세보증금이 가장 낮고, 서초구의 평균 전세보증금이 가장 높다.

In [77]:
roo_charter_for_geo_plot = room_charter[room_charter['보증금액'] < 50000].copy()
roo_charter_for_geo_plot['size'] = roo_charter_for_geo_plot['전용면적_m2'] ** 1.6

In [79]:
fig = px.scatter_mapbox(roo_charter_for_geo_plot, lat="random_lat", lon="random_long",
                         zoom=10, height=500, color='보증금액', size='size', opacity=0.3)
fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

**Geo plot**
- 노란색에 가까울수록 <font color="#CCCC00">전세보증금이 높고</font>, 파란색에 가까울수록 <font color="#0000BB">전세보증금이 낮음</font>
- 점의 크기가 클수록 전용면적($m^2$)이 큼

**About Plot**
- 강남, 서초, 강동구쪽이 대체로 전세보증금이 높다.
- 영등포구, 관악구, 강서구쪽이 대체로 전세보증금이 낮고, 전용면적도 넓다.